# LGBM - Rolling Windows

In [13]:
import numpy as np
import pandas as pd
from utils import *
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import sys
import xgboost as xgb
import lightgbm as lgb
from datetime import datetime

NUMBER_OF_LAGS = 4

sys.path.append("../../main/datasets/")
!ls  ../../main/datasets/

1.0v  1.0v.zip


<hr>

## Defining metrics

Baseline_score function

In [14]:
def baseline_score(prediction, target, simulatedPrice):
    prediction = prediction.astype(int)

    return np.sum((prediction - np.maximum(prediction - target, 0) * 1.6)  * simulatedPrice)

Evaluation Metric

In [15]:
def feval(prediction, dtrain):
    
    prediction = prediction.astype(int)
    target = dtrain.get_label()

    simulatedPrice = dtrain.get_weight()
    
    return 'feval', np.sum((prediction - np.maximum(prediction - target, 0) * 1.6)  * simulatedPrice), True

Objective Metric

In [16]:
def gradient(predt, dtrain):
    y = dtrain.get_label()
    sp = dtrain.get_weight()
    return -2 * (predt - np.maximum(predt - y, 0) * 1.6) * (1 - (predt > y) * 1.6) * sp

def hessian(predt, dtrain):
    y = dtrain.get_label()
    sp = dtrain.get_weight() 
    return -2 * ((1 - (predt > y) * 1.6) ** 2) * sp

def objective(predt, dtrain):
    grad = gradient(predt, dtrain)
    hess = hessian(predt, dtrain)
    return grad, hess

<hr>

## Building our dataset
This notebook makes this step cleaner than the previous versions. So It'll be tidier and shorter than before!

In [17]:
infos, items, orders = read_data("../../main/datasets/")
print("Sanity checks...", infos.shape, items.shape, orders.shape)

Sanity checks... (10463, 3) (10463, 8) (2181955, 5)


In [18]:
# Changing our time signatures
process_time(orders)

In [19]:
df = dataset_builder(orders, items)

<hr>

## Feature building

In [20]:
# percentage_accum_cat_3 feature...
df = cumulative_sale_by_category(df)

In [21]:
# df = time_encoder(df, 'group_backwards', 26)

In [22]:
df

,group_backwards,itemID,orderSum,brand,manufacturer,customerRating,category1,category2,category3,recommendedRetailPrice,percentage_accum_cat_3
0,13,1,0.0,0,1,4.38,1,1,1,8.84,0.000000
1,13,2,0.0,0,2,3.00,1,2,1,16.92,0.000000
2,13,3,1.0,0,3,5.00,1,3,1,15.89,0.000000
3,13,4,0.0,0,2,4.44,1,2,1,40.17,0.000000
4,13,5,2.0,0,2,2.33,1,1,1,17.04,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
136014,1,10459,0.0,180,253,0.00,8,44,8,56.57,0.001074
136015,1,10460,0.0,0,253,0.00,8,44,8,163.81,0.001074
136016,1,10461,0.0,0,253,0.00,8,44,8,128.01,0.000000
136017,1,10462,0.0,180,253,0.00,8,44,8,166.97,0.001074


In [23]:
# This cell lags and diffs our feature 'orderSum'
shifting = df.copy()

for i in range(1, NUMBER_OF_LAGS + 1):
    # Carrying the data of weeks t-1
    shifting[f'orderSum_{i}'] = shifting.groupby('itemID')['orderSum'].shift(i)

    
    # Getting the difference of the orders and promotions between weeks t-1 and t-2...
    shifting[f'orderSum_diff_{i}'] = shifting.groupby('itemID')[f'orderSum_{i}'].diff()
    
# LGBM Says on docs that it automatically handles zero values as NaN
shifting.fillna(0, inplace=True)

In [45]:
shifting.groupby(["itemID", 'group_backwards']).agg(orderSumMean=('orderSum','sum')).rolling(2).mean()

orderSumMean
itemID group_backwards              
1      1                         NaN
       2                         1.5
       3                        15.5
       4                        17.0
       5                       151.0
...                              ...
10463  9                         0.0
       10                        0.0
       11                        0.0
       12                        0.0
       13                        0.0

[136019 rows x 1 columns]

In [46]:
shifting.groupby(["itemID", 'group_backwards']).agg(orderSumMean=('orderSum','sum')).rolling(2).mean().reset_index().isna().sum()

itemID             0
group_backwards    0
orderSumMean       1
dtype: int64

<hr>

## Maximum error
The maximum error we could get in this dataset would be just guessing the mean of our sales from weeks 1 to 12, and that's what the cell below is computing.

In [34]:
worst_possible_prediction = shifting.loc[shifting.group_backwards < 13]['orderSum'].mean()
prediction = np.full(shifting.loc[shifting.group_backwards == 13]['orderSum'].shape, worst_possible_prediction) # Array filled with the mean...
target = shifting.loc[shifting.group_backwards == 13]['orderSum']
print("Guessing the mean of 'orderSum' for all items in target", mse(target, prediction) ** 0.5)

Guessing the mean of 'orderSum' for all items in target 90.29706562119341


<hr>

## Dataset Splitting
All my experiments will use weeks 13 to 3 as a train set, week 2 as our validation set and week 1 as a test set.

In [35]:
train = shifting.loc[shifting.group_backwards >= 3]
val = shifting.loc[shifting.group_backwards == 2]
test = shifting.loc[shifting.group_backwards == 1]

weights = infos.set_index('itemID')['simulationPrice'].to_dict()

w_train = train['itemID'].map(weights)
w_val = val['itemID'].map(weights)

In [36]:
# I recommend to the other members of the team keeping the
# datatypes of our datasets as Pandas DataFrames instead of Numpy,
# since It will easier to use Boosting Analysis frameworks
y_train = train['orderSum']
y_val = val['orderSum']
X_train = train.drop(columns=["orderSum"])
X_val = val.drop(columns=["orderSum"])

In [37]:
params = {
#           "objective" : "poisson",
          "objective" : "l1",
          "metric" :"rmse",
          "learning_rate" : 0.1,
          'verbosity': 1,
          'max_depth': 6,
          'num_leaves': 15,
          "min_data_in_leaf":2000,
         }

lgbtrain = lgb.Dataset(X_train, label = y_train, weight=w_train)
lgbvalid = lgb.Dataset(X_val, label = y_val, weight=w_val)

num_round = 1000
model = lgb.train(params,
                  lgbtrain,
                  num_round,
                  valid_sets = [lgbtrain, lgbvalid], 
                  verbose_eval=5,
                  early_stopping_rounds=5,
#                   fobj=objective,
                  feval=feval,
                  
                 )

Training until validation scores don't improve for 5 rounds
[5]	training's rmse: 39.9264	training's feval: 65277.1	valid_1's rmse: 44.9034	valid_1's feval: 9839.31
[10]	training's rmse: 39.9141	training's feval: 143268	valid_1's rmse: 44.891	valid_1's feval: 20604.1
[15]	training's rmse: 39.9097	training's feval: 167170	valid_1's rmse: 44.8883	valid_1's feval: 22434.4
Early stopping, best iteration is:
[11]	training's rmse: 39.9135	training's feval: 148278	valid_1's rmse: 44.8889	valid_1's feval: 22583.2


<hr>

### Utilities

**Predicting at test time**

In [121]:
y_test = test['orderSum']
X_test = test.drop(columns=["orderSum"])
final_predictions = model.predict(X_test)

In [122]:
final_predictions

array([0.69632997, 0.2       , 1.0079555 , ..., 0.        , 0.        ,
       0.        ])

In [123]:
final_predictions[final_predictions < 0] = 0

**Baseline calculation**

In [124]:
baseline_score(final_predictions, y_test.values, infos['simulationPrice'])

76860.294

**Creating our Kaggle CSV**

In [ ]:
final = pd.Series(0, index=np.arange(1, len(items)+1))
final[items.itemID] = final_predictions.astype(int)

final.to_csv("lgbm_kaggle_df.csv", header=["demandPrediction"],
            index_label="itemID", sep="|")

**Saving our model in disk**

In [ ]:
now = datetime.now().strftime("%d-%m-%Y-%Hh%Mm%Ss")
modelName = 'lgbm-' + now
xgb.save_model(modelName)